In [3]:
# Use autoplot 207's code for this app
import sys
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_dbconn, utc
import numpy as np

In [4]:
STORM_NUMBER = 15
TITLE = "30-31 January 2021"
SUBTITLE = "8 PM 31 January 2021"
sts = utc(2021, 1, 31, 6)
ets = utc(2021, 12, 30, 23, 0)
# Get available data
ctx = {'coop': 'yes', 'csector': 'IA', 't': 'state', 'sz': 30, 'z': 'yes', 'f': 'linear', 'v': 'snow'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx)
# add zeros and QC
df = add_zeros(df, ctx)

In [5]:
def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
                  "IEM 2020-2021 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    mp.contourf(
        lons, lats, vals, np.array([0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df_useme_plot)
    mp.drawcounties()
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'],
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=1)
    return mp

In [15]:
cull = []
if cull:
    df.loc[df['nwsli'].isin(cull), USEME] = False
# df.loc[df[df["state"] == "MN"].index, USEME] = False
hardcode = [
    [685, 3.5],
    ['WAUI4', 4],
    ['IA-BH-17', 4],
    ['IA-DV-14', 2],
    [1952, 0.8],
    
]
for nwsli, val in hardcode:
    df.loc[df['nwsli'] == nwsli, 'val'] = val

ctx['csector'] = "IA"
mp = workflow(ctx, df, isfinal=True, lower=1, upper=2)
res = mp.postprocess(filename='210131.png')
mp.close()

     state  wfo  val      lon      lat                              geo  \
1766    VA  LWX  1.9 -77.4500  38.8500  POINT (1925932.370 -419347.250)   
1767    WI  ARX  1.9 -91.6700  44.5600    POINT (658528.359 -15081.551)   
1768    VA  LWX  1.9 -77.4700  38.3000  POINT (1939760.509 -479415.190)   
1769    MD  LWX  1.9 -77.0900  39.0800  POINT (1949086.066 -385967.468)   
1770    IL  DVN  1.9 -89.1700  41.3500   POINT (901226.127 -346348.760)   
...    ...  ...  ...      ...      ...                              ...   
2717    PA  CTP  1.0 -78.5500  40.6500  POINT (1785593.654 -249001.681)   
2718    WI  DLH  1.0 -90.1953  46.4608    POINT (748741.199 208129.685)   
2719    WI  DLH  1.0 -90.9808  46.1694    POINT (692749.850 168860.784)   
2720    MI  MQT  1.0 -89.1794  46.3976    POINT (826715.305 211104.842)   
2721    IL  ILX  1.0 -89.0339  40.9125   POINT (918707.912 -393168.885)   

      used_for_analysis  nwsli  plotme source  
1766               True   1766    True    LSR  
176